In [9]:
def cut_dataset_between_twoday(data, first_day, end_day):
    data_part=data.loc[first_day-1<data['DAY']]
    data_part=data_part.loc[data_part['DAY']<end_day+1]
    data_part=data_part.reset_index(drop=True)
    
    return data_part


def cut_dataset_movement_reason(dataset, reason_transmission):
    market=["b'AGLOMERACAO COM FINALIDADE COMERCIAL'","b'AGLOMERACAO SEM FINALIDADE COMERCIAL'","b'RETORNO DE AGLOMERACAO'"]
    if reason_transmission=='Livestock_Market': #because there is 3 purpose related to livestock market
        data_part=dataset.loc[dataset['Finalidade'].isin(market)]
    else:
        data_part=dataset.loc[dataset['Finalidade']==reason_transmission]
    data_part=data_part.reset_index(drop=True)
    return data_part

    


# Generate network of premises

In [ ]:
def generate_net_premises_cityn_attr_datapart(data_part, w_method): 
  
    if w_method=='animal':
        edges_list=pd.DataFrame(data_part.groupby(['Codigo_do_Local_de_Origem','Codigo_do_Local_de_Destino',\
                    'city_n_origin','city_n_destination']).agg(weight=('animais','sum')).reset_index())
    if w_method=='movement':
        edges_list=pd.DataFrame(data_part.groupby(['Codigo_do_Local_de_Origem','Codigo_do_Local_de_Destino',\
                    'city_n_origin','city_n_destination']).size().reset_index(name="weight"))

    G=nx.DiGraph()
    for i in range(len(edges_list)):
        G.add_edge(edges_list['Codigo_do_Local_de_Origem'][i],\
        edges_list['Codigo_do_Local_de_Destino'][i],\
                    weight=edges_list['weight'][i]) #weight animal
        
    attr={}
    for j in range(len(edges_list)):
        attr[edges_list.loc[j,'Codigo_do_Local_de_Origem']]={'city_number':edges_list.loc[j,'city_n_origin']}
        attr[edges_list.loc[j,'Codigo_do_Local_de_Destino']]={'city_number':edges_list.loc[j,'city_n_destination']}
        
    nx.set_node_attributes(G, attr)
    
    return G


# Generate network of cities

In [ ]:
def generate_net_cities_weight_movement(dataframe, month_number, w_method, year_number):
    #w_method means make weight from sum of the number of movement or from sum of the number of animal 
    dict_year_month_day_number={'2013':{1:(0,30),2:(31,58),3:(59,89),4:(90,119),5:(120,150),6:(151,180),7:(181,211),\
        8:(212,242),9:(243,272),10:(272,303),11:(304,333),12:(334,364)},'2014':{1:(365,395),2:(396,423),3:(424,454),\
                4:(455,484),5:(485,515),6:(516,545),7:(546,576),8:(577,607),9:(608,637),10:(638,668),11:(669,698),\
                12:(699,729)},'2015':{1:(730,760),2:(761,788),3:(789,819),4:(820,849),5:(850,880),6:(881,910),\
                7:(911,941),8:(941,972),9:(973,1002),10:(1003,1033),11:(1034,1063),12:(1064,1094)},'2016':{1:(1095,1125)\
                ,2:(1126,1154),3:(1155,1185),4:(1186,1215),5:(1216,1246),6:(1247,1276),7:(1277,1307),8:(1308,1338)\
                ,9:(1339,1368),10:(1369,1399),11:(1400,1429),12:(1430,1459)}}

    #generate city_network
    data_nl=dataframe[dataframe['local_non_local']=='non_local']
    data_nl= data_nl.reset_index(drop=True)

    month_day=dict_year_month_day_number[year_number][month_number]
    #cut dataset for the 30 days for each month
    #using function named cut_dataset_between_twoday
    data_part= cut_dataset_between_twoday(data_nl, month_day[0], month_day[1])
    
    if w_method=='animal':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination'])\
                            .agg(weight=('animais','sum')).reset_index())
    if w_method=='movement':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination'])\
                            .size().reset_index(name="weight"))

    G_cities=nx.DiGraph()
    for i in range(len(edges_list)):
        G_cities.add_edge(edges_list['city_n_origin'][i],\
        edges_list['city_n_destination'][i],\
                    weight=edges_list['weight'][i]) #weight animal
    return G_cities



def generate_net_cities_yearly(dataframe, w_method, year_number):
    #w_method means make weight from sum of the number of movement or from sum of the number of animal 
    dict_year_month_day_number={'2013':{1:(0,30),2:(31,58),3:(59,89),4:(90,119),5:(120,150),6:(151,180),7:(181,211),\
        8:(212,242),9:(243,272),10:(272,303),11:(304,333),12:(334,364)},'2014':{1:(365,395),2:(396,423),3:(424,454),\
                4:(455,484),5:(485,515),6:(516,545),7:(546,576),8:(577,607),9:(608,637),10:(638,668),11:(669,698),\
                12:(699,729)},'2015':{1:(730,760),2:(761,788),3:(789,819),4:(820,849),5:(850,880),6:(881,910),\
                7:(911,941),8:(941,972),9:(973,1002),10:(1003,1033),11:(1034,1063),12:(1064,1094)},'2016':{1:(1095,1125)\
                ,2:(1126,1154),3:(1155,1185),4:(1186,1215),5:(1216,1246),6:(1247,1276),7:(1277,1307),8:(1308,1338)\
                ,9:(1339,1368),10:(1369,1399),11:(1400,1429),12:(1430,1459)}}

    #generate city_network
    data_nl=dataframe[dataframe['local_non_local']=='non_local']
    data_nl= data_nl.reset_index(drop=True)

    md1=dict_year_month_day_number[year_number][1]
    md2=dict_year_month_day_number[year_number][12]
    month_day=(md1[0],md2[1])
    #cut dataset for the 30 days for each month
    #using function named cut_dataset_between_twoday
    data_part= cut_dataset_between_twoday(data_nl, month_day[0], month_day[1])
    
    if w_method=='animal':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination'])\
                            .agg(weight=('animais','sum')).reset_index())
    if w_method=='movement':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination'])\
                            .size().reset_index(name="weight"))

    G_cities=nx.DiGraph()
    for i in range(len(edges_list)):
        G_cities.add_edge(edges_list['city_n_origin'][i],\
        edges_list['city_n_destination'][i],\
                    weight=edges_list['weight'][i]) #weight animal
    return G_cities

    
def generate_net_cities_datapart(data_part, w_method): 
    #include none local datframe
        #generate city_network
    data_part=data_part[data_part['local_non_local']=='non_local']
    data_nl= data_nl.reset_index(drop=True)
    
    if w_method=='animal':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination'])\
                            .agg(weight=('animais','sum')).reset_index())
    if w_method=='movement':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination'])\
                            .size().reset_index(name="weight"))

    G_cities=nx.DiGraph()
    for i in range(len(edges_list)):
        G_cities.add_edge(edges_list['city_n_origin'][i],\
        edges_list['city_n_destination'][i],\
                    weight=edges_list['weight'][i]) #weight animal
        

    return G_cities


# make network of one type of movement reason from 6 biggest movement reason

In [10]:
#function for make a network from sataset for one special movement reason

def make_movement_reason_network(dataset,reason_transmission, weight_type):
       
    data_part= cut_dataset_movement_reason(dataset, reason_transmission)
    #make network from this part of dataset include just one transmission reason
    if weight_type== 'movement':
        edges_list = pd.DataFrame(data_part.groupby(['Codigo_do_Local_de_Origem','Codigo_do_Local_de_Destino'])\
                                  .size().reset_index(name="weight"))
    if weight_type== 'animal':
        edges_list = pd.DataFrame(data_part.groupby(['Codigo_do_Local_de_Origem','Codigo_do_Local_de_Destino'])\
                    .agg(weight=('animais','sum')).reset_index())
    G = nx.from_pandas_edgelist(edges_list,'Codigo_do_Local_de_Origem','Codigo_do_Local_de_Destino', edge_attr='weight',create_using=nx.DiGraph())
    return G

def make_movement_reason_network_cities(dataset,reason_transmission, weight_type):

    data_part= cut_dataset_movement_reason(dataset, reason_transmission)
    #make network from this part of dataset include just one transmission reason
    
    if weight_type=='animal':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination'])\
                                .size().reset_index(name="weight"))
    if weight_type=='movement':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination'])\
                                                    .agg(weight=('animais','sum')).reset_index())

    G = nx.from_pandas_edgelist(edges_list,'city_n_origin','city_n_destination', edge_attr='weight',create_using=nx.DiGraph())
    return G


def make_movement_reason_network_atr_cityn(dataset,reason_transmission, weight_type):
    market=["b'AGLOMERACAO COM FINALIDADE COMERCIAL'","b'AGLOMERACAO SEM FINALIDADE COMERCIAL'","b'RETORNO DE AGLOMERACAO'"]
    if reason_transmission=='Livestock_Market': #because there is 3 purpose related to livestock market
        data_part=dataset.loc[dataset['Finalidade'].isin(market)]
    else:
        data_part=dataset.loc[dataset['Finalidade']==reason_transmission]
        
    data_part= data_part.reset_index(drop=True)
    #make network from this part of dataset include just one transmission reason
    if weight_type== 'movement':
        edges_list = pd.DataFrame(data_part.groupby(['Codigo_do_Local_de_Origem','Codigo_do_Local_de_Destino',\
                        'city_n_origin','city_n_destination'])\
                                  .size().reset_index(name="weight"))
    if weight_type== 'animal':
        edges_list = pd.DataFrame(data_part.groupby(['Codigo_do_Local_de_Origem','Codigo_do_Local_de_Destino',\
                     'city_n_origin','city_n_destination'])
                    .agg(weight=('animais','sum')).reset_index())
        
    G =nx.DiGraph()
    for i in range(len(edges_list)):
        G.add_edge(edges_list['Codigo_do_Local_de_Origem'][i],\
        edges_list['Codigo_do_Local_de_Destino'][i],\
                    weight=edges_list['weight'][i]) #weight animal
     
    attr={}
    for j in range(len(edges_list)):
        attr[edges_list.loc[j,'Codigo_do_Local_de_Origem']]={'city_number':edges_list.loc[j,'city_n_origin']}
        attr[edges_list.loc[j,'Codigo_do_Local_de_Destino']]={'city_number':edges_list.loc[j,'city_n_destination']}
        
    nx.set_node_attributes(G, attr)

    return G



def make_city_network_edge_label_movement_reason_MultiGraph(data_part, weight_type):
    
    if weight_type== 'movement':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination','Finalidade'])\
                                .size().reset_index(name="weight"))
    if weight_type== 'animal':
        edges_list=pd.DataFrame(data_part.groupby(['city_n_origin','city_n_destination','Finalidade'])\
                                                    .agg(weight=('animais','sum')).reset_index())
    
 
    G=nx.MultiDiGraph() 
    
    for i in range(len(edges_list)):
        G.add_edge(edges_list['city_n_origin'][i],\
             edges_list['city_n_destination'][i], \
        weight=edges_list['weight'][i], mov_reason=edges_list['Finalidade'][i])
        
    
    return G
        


# distribution

In [4]:
def dist_dataframe(list_t):
    #T_1SU=np.unique(list_t)  #if your list has none value this line will get error
    T_1SU=set(list_t)
    dist=[]# I use 'NM_MUNICIP' for column name to be similar to the column in MG geo data fram(dc)
    for x in T_1SU:
        C=list_t.count(x)
        dist.append((x, C))
    return dist



## Sort

In [2]:
def sort_pairlist_2nde_reverse(sub_li, sort_str):
  
    # reverse = None (Sorts in Ascending order)
    # key is set to sort using second element of 
    # sublist lambda has been used
    if sort_str=='sort':
        return(sorted(sub_li, key = lambda x: x[1])) 
    if sort_str=='reverse':
        return(sorted(sub_li, key = lambda x: x[1], reverse=True)) 

## Jaccaed similarity

In [3]:
def jaccard_set(list1, list2):
    """Define Jaccard Similarity function for two sets"""
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

# Map dataset

## dictionaries of city number city code and city center coordination

In [5]:
#dictionary from city number to city code
def dict_cityN_cityC():
    dataframe= pd.read_csv('Clean_dataset/clean_data_comprehensive_final.csv')
    dict_cityN_to_cityC={}
    list_pair= []
    for i in range(len(dataframe)):
        list_pair.append((dataframe.loc[i, 'city_n_origin'], dataframe.loc[i, 'Codigo_Municipio_de_Origem']))
        list_pair.append((dataframe.loc[i, 'city_n_destination'], dataframe.loc[i, 'Codigo_Municipio_de_Destino']))
    
    list_pair=list(set(list_pair))
    for i in range(len(list_pair)):
        dict_cityN_to_cityC[list_pair[i][0]]= list_pair[i][1]
    return  dict_cityN_to_cityC

def dict_cityC_cityN():
    dataframe= pd.read_csv('Clean_dataset/clean_data_comprehensive_final.csv')
    dict_cityN_to_cityC={}
    list_pair= []
    for i in range(len(dataframe)):
        list_pair.append((dataframe.loc[i, 'city_n_origin'], dataframe.loc[i, 'Codigo_Municipio_de_Origem']))
        list_pair.append((dataframe.loc[i, 'city_n_destination'], dataframe.loc[i, 'Codigo_Municipio_de_Destino']))
    
    list_pair=list(set(list_pair))
    for i in range(len(list_pair)):
        dict_cityN_to_cityC[list_pair[i][1]]= list_pair[i][0]
    return  dict_cityN_to_cityC

def dict_cityN_cityCoo():
    mg_ma=  gpd.read_file('mg_gps_map_data/mg_city/31MUE250GC_SIR.shp')
    mg_ma['center_co']= mg_ma['geometry'].centroid
    dict_cityc_cityn= dict_cityC_cityN()
    dict_cityN_to_cityCoo={}
    for i in range(len(mg_ma)):
        cn= dict_cityc_cityn[int(mg_ma.loc[i, 'CD_GEOCMU'])] 
        dict_cityN_to_cityCoo[cn] = mg_ma.loc[i, 'center_co']
    
    return dict_cityN_to_cityCoo 
        
############################################### Transformed Coordination #################################################
def dict_cityN_cityCoo_T():
    mg_ma=  gpd.read_file('mg_gps_map_data/mg_map_transform_87_118/mg_map_transform_87_118.shp')
    mg_ma['center_co']= mg_ma['geometry'].centroid
    dict_cityc_cityn= dict_cityC_cityN()
    dict_cityN_to_cityCoo={}
    for i in range(len(mg_ma)):
        cn= dict_cityc_cityn[int(mg_ma.loc[i, 'CD_GEOCMU'])] 
        dict_cityN_to_cityCoo[cn] = mg_ma.loc[i, 'center_co']
    
    return dict_cityN_to_cityCoo 
    
  
    
############################################### dictionary of city_number to miroregion name############################
def dict_cityNu_microNa(data_part):
    d1= dict(zip(data_part.city_n_origin, data_part.oigin_microregion_name))
    d2= dict(zip(data_part.city_n_destination, data_part.destination_microregion_name))
    d = {**d1, **d2}
    #d= z = d1 | d2
    #d= dict(d1.items() + d2.items())
    #remove duplicated items
    result = {}

    for key,value in d.items():
        if key not in result.keys():
            result[key] = value
            
    return result
    
    
############################################### dictionary of city_number to miroregion name############################
def dict_cityNu_mesoNa(data_part):
    d1= dict(zip(data_part.city_n_origin, data_part.oigin_mesoregion_name))
    d2= dict(zip(data_part.city_n_destination, data_part.destination_mesoregion_name))
    d = {**d1, **d2}
    #d= z = d1 | d2
    #d= dict(d1.items() + d2.items())
    #remove duplicated items
    result = {}

    for key,value in d.items():
        if key not in result.keys():
            result[key] = value
            
    return result   

def dict_micNa_micCoo():
    
    mg_mcrrgn_map=gpd.read_file('mg_gps_map_data/mcrrgn_mg/31MI2500G.shp')
    a = [mg_mcrrgn_map["geocodigo"], mg_mcrrgn_map["nome"],mg_mcrrgn_map['geometry']]
    headers = ["microregion_code", "name",'geometry']
    mg_mcrrgn_map = pd.concat(a, axis=1, keys=headers)
    mg_mcrrgn_map=gpd.GeoDataFrame(mg_mcrrgn_map)
    #assign number to microregion
    mg_mcrrgn_map['microregion_number']=mg_mcrrgn_map.index 
    mg_mcrrgn_map['center_co_mic']=mg_mcrrgn_map['geometry'].centroid
    dict_micna_miccoo={}
    for i in range(len(mg_mcrrgn_map)):
        dict_micna_miccoo[mg_mcrrgn_map.loc[i, 'name']] = mg_mcrrgn_map.loc[i, 'center_co_mic']
    
    return dict_micna_miccoo

def dict_micNu_micNa():
    
    mg_mcrrgn_map=gpd.read_file('mg_gps_map_data/mcrrgn_mg/31MI2500G.shp')
    a = [mg_mcrrgn_map["geocodigo"], mg_mcrrgn_map["nome"],mg_mcrrgn_map['geometry']]
    headers = ["microregion_code", "name",'geometry']
    mg_mcrrgn_map = pd.concat(a, axis=1, keys=headers)
    mg_mcrrgn_map=gpd.GeoDataFrame(mg_mcrrgn_map)
    #assign number to microregion
    mg_mcrrgn_map['microregion_number']=mg_mcrrgn_map.index 
    mg_mcrrgn_map['center_co_mic']=mg_mcrrgn_map['geometry'].centroid
    dict_micna_miccoo={}
    for i in range(len(mg_mcrrgn_map)):
        dict_micna_miccoo[mg_mcrrgn_map.loc[i, 'microregion_number']] = mg_mcrrgn_map.loc[i, 'name']
    
    return dict_micna_miccoo



In [11]:
#add values realated to the cities to gps dataset for ploting

def map_city_value(list_cityN_value):
#list_cityN_value: a list of pars. for each pair the first element is city number and second element 
#is value related to that city
    #dictionary of find the city code based on city number
    #dictionary[city_number]= city_code
    dict_cityN_to_cityC = np.load('mg_gps_map_data/dict_cityNunmber_to_cityCode.npy',allow_pickle='TRUE').item()
    
    #the dataframe of polygon data of cities
    mg_ma=gpd.read_file('mg_gps_map_data/mg_city/31MUE250GC_SIR.shp')
    
        
        
    
    for i in list_cityN_value:
        cc= dict_cityN_to_cityC[i[0]]
        for j in range(len(mg_ma)):
            if int(mg_ma.loc[j, 'CD_GEOCMU' ])== cc:
                mg_ma.loc[j, 'value']= i[1]
    return mg_ma
                
            
    

# Portrait Divergence

In [ ]:
#Functions are needed

def get_unique_path_lengths(graph, paths=None):
    if paths is None:
        paths = list(nx.all_pairs_dijkstra_path_length(graph))

    unique_path_lengths = set()
    for starting_node, dist_dict in paths:
        unique_path_lengths |= set(dist_dict.values())
    unique_path_lengths = sorted(list(unique_path_lengths))
    return unique_path_lengths


def get_portrait(graph):
#     giant = max(nx.connected_component_subgraphs(graph), key=len)
    diameter = 500 #nx.diameter(giant.to_undirected())
    N = graph.number_of_nodes()
    # B indices are 0...dia x 0...N-1:
    B = np.zeros((diameter + 1, N))

    max_path = 1
    adj = graph.adj
    for starting_node in graph.nodes():
        nodes_visited = {starting_node: 0}
        search_queue = [starting_node]
        d = 1
        while search_queue:
            next_depth = []
            extend = next_depth.extend
            for n in search_queue:
                l = [i for i in adj[n] if i not in nodes_visited]
                extend(l)
                for j in l:
                    nodes_visited[j] = d
            search_queue = next_depth
            d += 1

        node_distances = nodes_visited.values()
        max_node_distances = max(node_distances)

        curr_max_path = max_node_distances
        if curr_max_path > max_path:
            max_path = curr_max_path

        # build individual distribution:
        dict_distribution = dict.fromkeys(node_distances, 0)
        for d in node_distances:
            dict_distribution[d] += 1
        # add individual distribution to matrix:
        for shell, count in dict_distribution.items():
            B[shell][count] += 1

        # HACK: count starting nodes that have zero nodes in farther shells
        max_shell = diameter
        while max_shell > max_node_distances:
            B[max_shell][0] += 1
            max_shell -= 1

    return B[:max_path + 1, :]


def pad_portraits_to_same_size(b1, b2):
    ns, ms = b1.shape
    nl, ml = b2.shape

    # Bmats have N columns, find last *occupied* column and trim both down:
    last_col_1 = max(np.nonzero(b1)[1])
    last_col_2 = max(np.nonzero(b2)[1])
    last_col = max(last_col_1, last_col_2)
    b1 = b1[:, :last_col + 1]
    b2 = b2[:, :last_col + 1]

    big_b1 = np.zeros((max(ns, nl), last_col + 1))
    big_b2 = np.zeros((max(ns, nl), last_col + 1))

    big_b1[:b1.shape[0], :b1.shape[1]] = b1
    big_b2[:b2.shape[0], :b2.shape[1]] = b2

    return big_b1, big_b2


def zeros(shape, tCode=None):
    try:
        return np.zeros(shape, dtype=tCode)
    except TypeError:
        return np.zeros(shape, typecode='fd')  # hardwired to float


def graph_or_portrait(x):
    if isinstance(x, (nx.Graph, nx.DiGraph)):
        return get_portrait(x)
    return x


def element_wise_log(mat):
    new_mat = zeros(mat.shape, tCode=float)
    i = 0
    for row in mat:
        j = 0
        for e in row:
            if e != 0:
                new_mat[i, j] = log(e + 1)
            else:
                new_mat[i, j] = 0
            j += 1
        i += 1
    return new_mat


def create_matrix_plot(o_mat, title, **kwargs):
    kwargs['interpolation'] = 'nearest'
    origin = kwargs.get('origin', 1)
    kwargs['origin'] = 'lower'
    showColorBar = kwargs.get('showColorBar', False)
    if "showColorBar" in kwargs: kwargs.pop("showColorBar")
    logColors = kwargs.get('logColors', False)
    if "logColors" in kwargs: kwargs.pop("logColors")
    ifShow = kwargs.get('show', False)
    if "show" in kwargs: kwargs.pop("show")
    fileName = kwargs.get('fileName', None)
    if "fileName" in kwargs: kwargs.pop("fileName")

    mat = o_mat.copy()  # don't modify original matrix
    if logColors: mat = element_wise_log(mat)

    cmap = plt.cm.seismic
    cmap.set_under(color='white')

    if "vmax" not in kwargs:
        kwargs['vmax'] = float(mat[origin:, origin:].max())
        kwargs['vmin'] = float(mat[origin:, origin:].min() + sys.float_info.epsilon)
        kwargs['cmap'] = cmap

    ax = plt.axes()  # [.05,.05,.9,.9])
    ax.xaxis.tick_top()
    h = plt.imshow(mat, **kwargs)
    plt.title(title)
    plt.axis('tight')
    # ax.set_xlim((origin, 10))
    ax.set_ylim((mat.shape[0], origin))

    if showColorBar: plt.colorbar()

    if fileName is not None:
        plt.savefig(fileName)
        
    if ifShow:    
        plt.show()

    return h


def portrait_divergence(g, h):
    bg = graph_or_portrait(g)
    bh = graph_or_portrait(h)
    bg, bh = pad_portraits_to_same_size(bg, bh)

    l, k = bg.shape
    v = np.tile(np.arange(k), (l, 1))

    xg = bg * v / (bg * v).sum()
    xh = bh * v / (bh * v).sum()

    # flatten distribution matrices as arrays:
    p = xg.ravel()
    q = xh.ravel()

    # lastly, get JSD:
    m = 0.5 * (p + q)
    kld_pm = entropy(p, m, base=2)
    kld_qm = entropy(q, m, base=2)
    jsd_pq = 0.5 * (kld_pm + kld_qm)

    return jsd_pq, bg, bh


def weighted_portrait(g, paths=None, bin_edges=None):
    from collections import Counter
    if paths is None:
        paths = list(nx.all_pairs_dijkstra_path_length(g))

    if bin_edges is None:
        unique_path_lengths = get_unique_path_lengths(g, paths=paths)
        sampled_path_lengths = np.percentile(unique_path_lengths, np.arange(0, 101, 1))
    else:
        sampled_path_lengths = bin_edges
    upl = np.array(sampled_path_lengths)

    l_s_v = []
    for i, (s, dist_dict) in enumerate(paths):
        distances = np.array(list(dist_dict.values()))
        s_v, e = np.histogram(distances, bins=upl)
        l_s_v.append(s_v)
    m = np.array(l_s_v)

    b = np.zeros((len(upl) - 1, g.number_of_nodes() + 1))
    for i in range(len(upl) - 1):
        col = m[:, i]  # ith col = numbers of nodes at d_i <= distance < d_i+1
        for n, c in Counter(col).items():
            b[i, n] += c

    return b


def portrait_divergence_weighted(g, h, bins=None, bin_edges=None):
    from collections import Counter
    from scipy.stats import entropy
    # get joint binning:
    paths_g = list(nx.all_pairs_dijkstra_path_length(g))
    paths_h = list(nx.all_pairs_dijkstra_path_length(h))

    # get bin_edges in common for G and H:
    if bin_edges is None:
        if bins is None:
            bins = 1
        upl_g = set(get_unique_path_lengths(g, paths=paths_g))
        upl_h = set(get_unique_path_lengths(h, paths=paths_h))
        unique_path_lengths = sorted(list(upl_g | upl_h))
        bin_edges = np.percentile(unique_path_lengths, np.arange(0, 101, bins))

    # get weighted portraits:
    bg = weighted_portrait(g, paths=paths_g, bin_edges=bin_edges)
    bh = weighted_portrait(h, paths=paths_h, bin_edges=bin_edges)

    return portrait_divergence(bg, bh)
